#### Importing necessary libraries

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # This avoids SettingWithCopyWarnings
import numpy as np



In [2]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

### Initializing Variables


In [3]:
data={} # Dictionary to contain pandas dataframe for all the stocks. This is to avoid creating variable for each stock 
        # to store data
finalData={} # This should contain our final output and that is Renko OHLC data
n=7 # Period for ATR
renkoData={} # It contains information on the lastest bar of renko data for the number of stocks we are working on

### Average True Range is generally used as Renko bar size. However we can also use a custom bar size

In [4]:
def ATR(df,n): #df is the DataFrame, n is the period 7,14 ,etc
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.ix[n-1,'ATR']=df['TR'][:n-1].mean() #.ix is deprecated from pandas version- 0.19
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n
    return 

### Set of Stocks

In [5]:
StockList=['INFY', 'ICICIBANK'] 

### Fetching data using Yahoo API,  Setting up Pandas Dataframe in the dictionary 'data' with key as Stock name and Value as DataFrame


In [6]:
start='2016-1-1'
for stock in StockList:
    data[stock]=pdr.get_data_yahoo(stock+'.NS', start)

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


#### Sometimes due to rate limiting factor, Yahoo api when called yeilds no data. Below cell helps us out in figuring whether we need to ping the api again. 

In [7]:
for stock in data:
    if data[stock].empty:
        print (stock)

In [8]:
print(data)

{'INFY':                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-01-01  550.000000  554.474976  546.599976  552.625000  477.152191   
2016-01-04  549.974976  551.224976  538.025024  539.450012  465.776550   
2016-01-05  542.750000  542.750000  531.125000  537.025024  463.682709   
2016-01-06  537.325012  537.325012  529.450012  534.674988  461.653625   
2016-01-07  530.025024  533.950012  523.950012  525.400024  453.645386   
2016-01-08  530.750000  533.700012  527.750000  531.650024  459.041748   
2016-01-11  528.150024  534.974976  526.049988  527.849976  455.760742   
2016-01-12  531.000000  531.474976  515.424988  524.974976  453.278351   
2016-01-13  528.500000  545.500000  524.049988  541.700012  467.719269   
2016-01-14  533.000000  577.500000  527.174988  566.500000  489.132233   
2016-01-15  574.000000  582.000000  568.775024  569.950012  492.111084   
2016-01-18  569.950012  581.0

###  Data Cleaning, Column renaming, etc

In [9]:
for stock in data:
    data[stock].drop(data[stock][data[stock].Volume == 0].index, inplace=True) # Data Cleaning
    ATR(data[stock],n)
    data[stock]=data[stock][['Open','High','Low','Close','ATR']] # Removing unwanted columns

/Users/chris/Projects/RenkoBot/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [10]:
data['INFY'].head() #Since ATR period is 7, we will see atr column filling out from 7th row

,Open,High,Low,Close,ATR
Date,,,,,
2016-01-01,550.000000,554.474976,546.599976,552.625000,NaN
2016-01-04,549.974976,551.224976,538.025024,539.450012,NaN
2016-01-05,542.750000,542.750000,531.125000,537.025024,NaN
2016-01-06,537.325012,537.325012,529.450012,534.674988,NaN
2016-01-07,530.025024,533.950012,523.950012,525.400024,NaN


In [11]:
data['ICICIBANK'].head(10)

,Open,High,Low,Close,ATR
Date,,,,,
2016-01-01,237.544998,239.636002,234.544998,239.091003,NaN
2016-01-04,237.272995,237.591003,231.500000,232.317993,NaN
2016-01-05,232.955002,234.091003,228.817993,233.363998,NaN
2016-01-06,232.182007,233.408997,226.544998,227.363998,NaN
2016-01-07,224.000000,225.182007,221.000000,224.317993,NaN
2016-01-08,222.272995,225.272995,221.317993,222.772995,NaN
2016-01-11,219.908997,221.727005,216.408997,217.682007,5.856336
2016-01-12,219.091003,219.863998,214.182007,215.227005,5.831430
2016-01-13,216.182007,220.817993,210.091003,217.727005,6.530795


### Initializing Renkodata Dictionary which will contain Renko BrickSize, Open, Close and its Color

In [12]:
for stock in data:
    renkoData[stock]={'BrickSize':0.0, 'Open':0.0,'Close':0.0,'Color':''}

### Setting up Bricksize from the last row of ATR column or it can be set manually as well with custom size 

In [13]:
for stock in data:
    renkoData[stock]['BrickSize']=round(data[stock]['ATR'][-1],2) #This can be set manually as well!
    renkoData[stock]['Open']=renkoData[stock]['BrickSize']+renkoData[stock]['Close'] # This can be done the otherway round
                                                                                    # as well.'Close' = 'BrickSize' - 'Open' 
    renkoData[stock]['Color']='red'    # Should you choose to do the other way round, please change the color to 'green'

In [14]:
renkoData['ICICIBANK']

{'BrickSize': 9.79, 'Open': 9.79, 'Close': 0.0, 'Color': 'red'}

In [15]:
renkoData['INFY']

{'BrickSize': 18.74, 'Open': 18.74, 'Close': 0.0, 'Color': 'red'}

 ### Setting us the finalData dictionary which will contain  Renko OHLC data along with the bar color!

In [16]:
for stock in data:
    finalData[stock]=pd.DataFrame()
    finalData[stock].index.name='Date'
    finalData[stock]['ReOpen']=0.0
    finalData[stock]['ReHigh']=0.0
    finalData[stock]['ReLow']=0.0
    finalData[stock]['ReClose']=0.0
    finalData[stock]['Color']=''


### The calculation involves searching for Renko bars as and when prices close above or below the previous renko bars and recording it 

In [17]:
for stock in data: # This loops thorugh all the stocks in the data dictionary
    for index,row in data[stock].iterrows(): # One may choose to use Pure python instead of Iterrows to loop though each n 
                                         # every row to improve performace if datasets are large.
        if renkoData[stock]['Open']> renkoData[stock]['Close']: 
            while row['Close']> renkoData[stock]['Open']+renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']+=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']+=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Close']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['Color'].loc[index]='green'

            while row['Close'] < renkoData[stock]['Close']-renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']-=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']-=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Open']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['Color'].loc[index]='red'
                
        else:
            while row['Close']< renkoData[stock]['Open']-renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']-=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']-=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Close']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['Color'].loc[index]='red'
                
            while row['Close'] > renkoData[stock]['Close']+renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']+=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']+=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Open']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['Color'].loc[index]='green'



In [18]:
finalData['ICICIBANK'].tail()

,ReOpen,ReHigh,ReLow,ReClose,Color
Date,,,,,
2018-09-12,332.86,332.86,323.07,323.07,red
2018-09-24,323.07,323.07,313.28,313.28,red
2018-10-29,332.86,342.65,332.86,342.65,green
2018-10-31,342.65,352.44,342.65,352.44,green
2018-11-14,352.44,362.23,352.44,362.23,green


In [19]:
finalData['INFY'].tail()

,ReOpen,ReHigh,ReLow,ReClose,Color
Date,,,,,
2018-10-11,693.38,693.38,674.64,674.64,red
2018-10-24,674.64,674.64,655.90,655.90,red
2018-10-26,655.90,655.90,637.16,637.16,red
2018-10-31,655.90,674.64,655.90,674.64,green
2018-11-21,655.90,655.90,637.16,637.16,red


#### Taking data to excel sheet

In [22]:
writer = pd.ExcelWriter('Data.xlsx')
for stock in data:
    data[stock].to_excel(writer,stock)
writer.save()

writer = pd.ExcelWriter('RenkoData.xlsx')
for stock in finalData:
    finalData[stock].to_excel(writer,stock)
writer.save()


NameError: name 'file' is not defined